<a href="https://colab.research.google.com/github/VinayRawat2004/IMBD-NLP-/blob/main/imbdNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

THE FIRST STEP IS CHANGING THE RUNTIME TYPE FROM CPU TO GPU
T4 GPU AS I WILL USE NEURAL NETWORK ALGO (LSTM)

In [27]:
!pip install kaggle

DEPENDENCIES IMPORT, USING KAGGLE LIBRARY BECAUSE OF THE DATSET I USED

In [28]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



*   DATA COLLECTION
*   FROM KAGGLE API : json.laod converts json object into a python dictionary



In [29]:
kag_api = json.load(open("kaggle.json"))

set kaggle credentials as environmental variable, dont directly insert the key as if you share your file with someone

In [30]:
os.environ["KAGGLE_USERNAME"] = kag_api["username"]
os.environ["KAGGLE_KEY"] = kag_api["key"]

In [31]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [32]:
!ls


'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


UNZIP file

In [33]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip","r") as zip:
  zip.extractall()

In [34]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


LOAD DATA

In [35]:
data = pd.read_csv("IMDB Dataset.csv")

In [36]:
data.shape

(50000, 2)

In [37]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [38]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


### **EDA**

In [39]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [40]:
data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

<ipython-input-40-329ec005e21e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)


In [41]:
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [42]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [43]:
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


SPLIT THE DATA INTO TEST AND TARAIN DATA

In [44]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [48]:
train_data.shape

(40000, 2)

In [49]:
test_data.shape

(10000, 2)

**DATA PREPROCESS STEPS**


1.   TOKENIZE TEXT DATA
2.   List item



In [50]:
tok= Tokenizer(num_words=5000)
tok.fit_on_texts(train_data['review'])
X_train = pad_sequences(tok.texts_to_sequences(train_data["review"]),maxlen=200)
X_test = pad_sequences(tok.texts_to_sequences(test_data["review"]),maxlen=200)

In [53]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [55]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


IT DOESNT HAVE THE SENTIMENT OR OUTPUT, LET'S MAKE TWO MORE

In [56]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [57]:
print(Y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


In [58]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


# LSTM

In [64]:
model = Sequential()  #stack layers
model.add(Embedding(input_dim=5000, output_dim=128,input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [67]:
model.build(input_shape=(None, 200))
print(model.summary())


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

None


In [70]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

TRAIN THE MODEL

In [72]:
model.fit(X_train, Y_train, batch_size=64, epochs=8, validation_split=0.2)


Epoch 1/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 129s 257ms/step - accuracy: 0.8582 - loss: 0.3446 - val_accuracy: 0.8671 - val_loss: 0.3132
Epoch 2/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 126s 252ms/step - accuracy: 0.8585 - loss: 0.3356 - val_accuracy: 0.8748 - val_loss: 0.3083
Epoch 3/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 253ms/step - accuracy: 0.8872 - loss: 0.2810 - val_accuracy: 0.8568 - val_loss: 0.3441
Epoch 4/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 143s 255ms/step - accuracy: 0.8969 - loss: 0.2581 - val_accuracy: 0.8796 - val_loss: 0.3017
Epoch 5/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 255ms/step - accuracy: 0.9195 - loss: 0.2100 - val_accuracy: 0.8698 - val_loss: 0.3385
Epoch 6/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 128s 257ms/step - accuracy: 0.9224 - loss: 0.1961 - val_accuracy: 0.8749 - val_loss: 0.3426
Epoch 7/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 127s 254ms/step - accuracy: 0.9401 - loss: 0.1600 - val_accuracy: 0.8773 - val_loss: 0.3294
Epoch 8/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 143s 257ms/step - accuracy: 0.9497 - loss: 0

MODEL EVALUATION

In [74]:
loss, accuracy=model.evaluate(X_test, Y_test)
print("TEST LOSS:",loss)
print("TEST ACCURACY:",accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 82ms/step - accuracy: 0.8779 - loss: 0.3469
TEST LOSS: 0.3430485725402832
TEST ACCURACY: 0.8819000124931335


PREDICTION SYSTEM

In [75]:
def predict_sentiment(review):
  # tokenize and then pad the review
  sequence = tok.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

EXAMPLES

In [76]:
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
The sentiment of the review is: positive


In [78]:
new_review = "This movie was not so good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
The sentiment of the review is: negative


In [79]:
new_review = "Did not like the characters, movie was not so good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
The sentiment of the review is: negative


In [81]:
new_review = "this was hands down the best movie I’ve ever seen in my life. But, there were a couple of moments where the pacing dragged a bit. Still, it didn’t take away from how amazing it was overall"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
The sentiment of the review is: positive


In [82]:
new_review = "I really wanted to like this movie, but it felt like a missed opportunity. The performances were top-notch, but the story was flat, and the direction was off. It’s just not what I expected."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
The sentiment of the review is: negative


In [87]:
new_review = "movie was genuinely good,the charcters though were bad"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
The sentiment of the review is: positive


In [88]:
model.save('imdb_sentiment_model.h5')  # Save the model in HDF5 format

# To download the model to your local machine
from google.colab import files
files.download('imdb_sentiment_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>